## Cython/Python of the Bernoulli Distribution with MultiMin

Here we will program the above cells in Python and call a Monte-Carlo function to yield us the outputs we desire. The Monte-Carlo function will be written in Cython and we will use the %timeit cell magic to give us a run-time of our Cython function. 

Here we are forced to use MultiMin because the behaviour of the Bernoulli log likelihood function does not behave in such a way that a minimum can be found using the normal univariate minimization routine. Moreover, we are forced to transform the probability to lie within [0,1] given the use of logs. 

### Cython Function

In [1]:
%load_ext cython

In [2]:
%%cython -lgsl -lgslcblas

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True

#Import modules: 

from libc.stdlib cimport rand, RAND_MAX, malloc, realloc, free, abort
from libc.math cimport log, sin

#Use the CythonGSL package to get the low-level routines
from cython_gsl cimport *

######################### Define the Data Structure ############################

cdef struct Parameters:
    #Pointer for Y data array
    double* Y
    #size of the array
    int* Size
    
############################# Global Variables ################################

cdef double bounds[2]

################ Support Functions for Monte-Carlo Function ##################

#Create a function that allocates the memory and verifies integrity
cdef int alloc_struct(Parameters* data, int* N, unsigned int flag) nogil:
    
    cdef int Mem_Int = True
    
    #fill in the size of the array
    data.Size = N
    
    #allocate the data array initially
    if flag==0:
        data.Y = <double*> malloc(N[0] * sizeof(double))
    #reallocate the data array
    else:
        data.Y = <double*> realloc(data.Y, N[0] * sizeof(double))
    
    #If the elements of the struct are not properly allocated, destory it and return null
    if N[0]!=0 and data.Y==NULL:
        
        #return the memory to system
        destroy_struct(data)
        
        #update the memory integrity variable to False
        Mem_Int = False
    
    return Mem_Int

#Create the destructor of the struct to return memory to system
cdef void destroy_struct(Parameters* data) nogil:
    free(data.Y)
    free(data)

#This function fills in the Y observed variable with discreet 0/1
cdef void Y_fill(Parameters* data, double p_true) nogil:
    
    cdef:
        Py_ssize_t i
        double y
    
    for i in range(data.Size[0]):
        
        y = rand()/<double>RAND_MAX
        
        if y <= p_true:
            data.Y[i] = 1.0
        else: data.Y[i] = 0.0

#Definition of the function to be maximized: LLF of Bernoulli
cdef double LLF(gsl_vector* v, void* data) nogil:
    
    cdef:
        #the sample structure (considered the parameter here), recast
        Parameters* sample = <Parameters*> data
        
        #the loop variables
        Py_ssize_t i
        int n = sample.Size[0]
        
        #other variables
        double Sum = 0, p = gsl_vector_get(v, 0), p_tran
    
    #Transform variable to bound the value between (0,1)
    p_tran = ((bounds[0] + bounds[1])/2) + ((bounds[0] - bounds[1])/2)*sin(p)
    
    for i in range(n):
        
        if sample.Y[i]==1.0: Sum += log(p_tran)
        else: Sum += log(1-p_tran)
    
    return (-(Sum/n))

########################## Monte-Carlo Function ##############################

cpdef void Monte_Carlo(int[::1] Samples, double[:,::1] p_hat, 
                       int Sims, double p_true) nogil:
     
    #Define variables and pointers
    
    global bounds
    
    bounds[0] = 0; bounds[1] = 1
    
    cdef:
        #Data Structure
        Parameters* Data
            
        #iterators
        Py_ssize_t i, j, k
        int status, Iter, max_Iter = 1000 
        
        #Variables
        int N = Samples.shape[0], Mem_Int
        double tol = 1e-6, size
        
        #GSL objects and pointer
        const gsl_multimin_fminimizer_type* T
        gsl_multimin_fminimizer* s
        gsl_multimin_function F
        gsl_vector *ss
        gsl_vector *x
    
    #Allocate the minimization routine (multimin)
    T = gsl_multimin_fminimizer_nmsimplex
    s = gsl_multimin_fminimizer_alloc(T,1)
    
    #Allocate the Step Size and variable vector
    ss = gsl_vector_alloc(1)
    x = gsl_vector_alloc(1)
    
    #allocate the struct dynamically
    Data = <Parameters*> malloc(sizeof(Parameters))
    
    #verify memory integrity
    if Data==NULL or s==NULL or ss==NULL or x==NULL: abort()
        
    #Set the GSL function
    F.n = 1
    F.f = &LLF
    F.params = <void*> Data
    
    #Allocate the Step Size vector
    gsl_vector_set(ss,0,0.5)
    
    try:
        
        for i in range(N): 

            #allocate the elements of the struct (if i>0, reallocate data array)
            Mem_Int = alloc_struct(Data, &Samples[i], i)

            #verify memory integrity
            if Mem_Int==False: abort() 

            for j in range(Sims):

                #fill the array in the struct
                Y_fill(Data, p_true)
                
                #set the starting value (random number)
                gsl_vector_set(x, 0, rand()/RAND_MAX)
               
                #set the minimizer
                gsl_multimin_fminimizer_set(s, &F, x, ss)

                #initialize conditions
                Iter = 0
                status = GSL_CONTINUE

                while (status == GSL_CONTINUE and Iter <= max_Iter):
                    
                    Iter += 1
                    status = gsl_multimin_fminimizer_iterate(s)

                    if status:
                        break

                    size = gsl_multimin_fminimizer_size(s)
                    status = gsl_multimin_test_size(size, tol)

                    if status==GSL_SUCCESS:
                        p_hat[i,j] = gsl_vector_get(s.x,0)

    finally:
        destroy_struct(Data)
        gsl_multimin_fminimizer_free(s)
        gsl_vector_free(x)
        gsl_vector_free(ss)

### Python Portion and Cython Function Call

In [3]:
import numpy as np

In [4]:
#First we will recreate the first cell in the matlab example in python

#Sample Sizes
N = np.array([5,50,500,5000], dtype='i')

#Parameters for MC
T = 1000
p_true = 0.2

#Array of the outputs from the MC
p_hat = np.empty((N.size,T), dtype='d')
p_hat.fill(np.nan)

In [5]:
%timeit Monte_Carlo(N, p_hat, T, p_true)

1 loop, best of 3: 1.96 s per loop


In [7]:
p_hat.mean(axis=1)

array([ 0.80281857,  0.64867009,  0.64455261,  0.6431447 ])